# Rough Alignment 2: Diagnostic Plot

In [1]:
from pathlib import Path

import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import numpy as np
import SimpleITK as sitk

import utility

## Load Images and Transformations

In [2]:
mov_brain = 'DK52'
fix_brain = 'DK43'
thumb_spacing = (10.4, 10.4, 20.0)

In [3]:
data_dir = Path('/net/birdstore/Active_Atlas_Data/data_root/pipeline_data')
mov_img_thumb_dir = data_dir / mov_brain / 'preps/CH1/thumbnail_aligned'
fix_img_thumb_dir = data_dir / fix_brain / 'preps/CH1/thumbnail_aligned'
affine_save_path = f'../data/automatic-alignment/{fix_brain}/1-affine.tfm'
demons_save_path = f'../data/automatic-alignment/{fix_brain}/1-demons.tfm'

In [4]:
%%time
moving_image = utility.load_image_dir(mov_img_thumb_dir, spacing=thumb_spacing)

CPU times: user 18.3 s, sys: 5.78 s, total: 24.1 s
Wall time: 18.8 s


In [5]:
%%time
fixed_image = utility.load_image_dir(fix_img_thumb_dir, spacing=thumb_spacing)

CPU times: user 12.9 s, sys: 3.14 s, total: 16 s
Wall time: 13.2 s


In [6]:
%%time
affine_transform = sitk.ReadTransform(affine_save_path)

CPU times: user 0 ns, sys: 811 µs, total: 811 µs
Wall time: 523 µs


In [7]:
%%time
demons_transform = sitk.ReadTransform(demons_save_path)

CPU times: user 27.6 s, sys: 2.43 s, total: 30 s
Wall time: 30 s


## Apply Transformations

In [8]:
%%time
affine_transformed_image = sitk.Resample(
    moving_image, fixed_image, affine_transform,
    sitk.sitkLinear, 0.0, moving_image.GetPixelID()
)

CPU times: user 42.7 s, sys: 3.87 s, total: 46.6 s
Wall time: 3.24 s


In [9]:
%%time
demons_transformed_image = sitk.Resample(
    moving_image, fixed_image, demons_transform,
    sitk.sitkLinear, 0.0, moving_image.GetPixelID()
)

CPU times: user 3min 22s, sys: 3.06 s, total: 3min 25s
Wall time: 14.2 s


## Plot & Save

In [10]:
z_step = 10

In [11]:
# convert all images to arrays
fixed_arr = sitk.GetArrayViewFromImage(fixed_image)
affine_transformed_arr = sitk.GetArrayViewFromImage(affine_transformed_image)
demons_transformed_arr = sitk.GetArrayViewFromImage(demons_transformed_image)

### General Diagnostic Plots

In [12]:
save_path = f'../data/automatic-alignment/{fix_brain}/1-diagnostics.pdf'
figure_kwargs = {
    'dpi': 200,
    'figsize': (8, 6),
}
imshow_kwargs = {
    'aspect':'equal',
    'cmap': 'gray',
}
with PdfPages(save_path) as pdf:
    sz = fixed_arr.shape[0]
    for z in range(0, sz, z_step):
        print(f'{z}/{sz}', end='\r')
        fig, ax = plt.subplots(1, 3, **figure_kwargs)
        ax[0].imshow(affine_transformed_arr[z,:,:].T, **imshow_kwargs)
        ax[0].set_title(f'{mov_brain} after Affine')
        ax[0].set_axis_off()
        ax[1].imshow(demons_transformed_arr[z,:,:].T, **imshow_kwargs)
        ax[1].set_title(f'{mov_brain} after Demons')
        ax[1].set_axis_off()
        ax[2].imshow(fixed_arr[z,:,:].T, **imshow_kwargs)
        ax[2].set_title(f'{fix_brain} Fixed')
        ax[2].set_axis_off()
        fig.suptitle(f'z = {z}')
        fig.tight_layout()
        pdf.savefig(fig)
        plt.close()
print('Finished!')

Finished!


### Alternating Plots

In [13]:
save_path = f'../data/automatic-alignment/{fix_brain}/1-diagnostics-alt.pdf'
figure_kwargs = {
    'dpi': 200,
    'figsize': (8, 6),
}
imshow_kwargs = {
    'aspect':'equal',
    'cmap': 'gray',
}
with PdfPages(save_path) as pdf:
    sz = fixed_arr.shape[0]
    for z in range(0, sz, z_step):
        print(f'{z}/{sz}', end='\r')
        
        fig = plt.figure(**figure_kwargs)
        plt.imshow(fixed_arr[z,:,:], **imshow_kwargs)
        plt.title(f'z = {z}\n{fix_brain} Fixed')
        plt.axis('off')
        pdf.savefig(fig)
        plt.close()

        fig = plt.figure(**figure_kwargs)
        plt.imshow(affine_transformed_arr[z,:,:], **imshow_kwargs)
        plt.title(f'z = {z}\n{mov_brain} after Affine')
        plt.axis('off')
        pdf.savefig(fig)
        plt.close()

        fig = plt.figure(**figure_kwargs)
        plt.imshow(demons_transformed_arr[z,:,:], **imshow_kwargs)
        plt.title(f'z = {z}\n{mov_brain} after Demons')
        plt.axis('off')
        pdf.savefig(fig)
        plt.close()

        fig = plt.figure(**figure_kwargs)
        plt.imshow(fixed_arr[z,:,:], **imshow_kwargs)
        plt.title(f'z = {z}\n{fix_brain} Fixed')
        plt.axis('off')
        pdf.savefig(fig)
        plt.close()
print('Finished!')

Finished!
